In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
import joblib
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import SlidingWindows as data

2024-06-11 09:20:21.579368: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-11 09:20:21.612177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 09:20:21.612198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 09:20:21.613193: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-11 09:20:21.618928: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#Se obtienen los datos de los sensores y las etiquetas deducidas
print("Obteniendo datos...")
data_2206_17_bag, tags_220617_bag = data.get_data('/home/cmonedero/TFG/User1/Motion/User1_220617_Bag.csv', '/home/cmonedero/TFG/User1/220617/Label_New_User1_220617.txt')

Obteniendo datos...


Hecho!



In [3]:
#Se divide el dataset en ventanas de tamaño 500
print("Dividiendo en ventanas el dataset...")
x_windows_220617_bag, y_windows_220617_bag = data.divide_data_arrays_into_windows(data_2206_17_bag, tags_220617_bag, window_size=500, moda=True)
# print("Tamaño ventanas X --> ", x_windows_220617_bag.shape)
# print("Tamaño ventanas Y --> ", y_windows_220617_bag.shape)

Dividiendo en ventanas el dataset...
Hecho!



In [4]:
print("Dividiendo en train y test...")
x_train_220617_bag, y_train_220617_bag, x_test_220617_bag, y_test_220617_bag = data.divide_train_test(x_windows_220617_bag, y_windows_220617_bag, 0.8)
print("Tamaño x_train --> ", x_train_220617_bag.shape)
print("Tamaño y_train -->", y_train_220617_bag.shape)
print("Tamaño x_test --> ", x_test_220617_bag.shape)
print("Tamaño y_test -->", y_test_220617_bag.shape)

# x_tf_220617_bag = tf.data.Dataset.from_tensor_slices((tf.constant(x_train_220617_bag), tf.cast(tf.constant(y_train_220617_bag), tf.float32)))
# BATCH_SIZE=32
# x_tf_train_220617_bag = x_tf_220617_bag.cache().batch(BATCH_SIZE)
# print(type(x_tf_train_220617_bag))
# print(x_tf_train_220617_bag)

Dividiendo en train y test...
Hecho!

Tamaño x_train -->  (2920, 500, 6)
Tamaño y_train --> (2920,)
Tamaño x_test -->  (731, 500, 6)
Tamaño y_test --> (731,)


In [5]:
#SOLO EJECUTAR ESTA CELDA SI SE QUIERE VOLVER A ENTRENAR EL MODELO O SE ENTRENA POR PRIMERA VEZ!!!
input_shape = x_train_220617_bag.shape[1:]
print(input_shape)
print("Creando el modelo...")
model_220617_bag = data.create_model(input_shape)
print(model_220617_bag.summary())
cp_220617_bag = ModelCheckpoint('/home/cmonedero/TFG/MHA/checkpoint.model.keras', save_best_only=True)
#print(x_tf_train_220617_bag)
# data.train_model(model_220617_bag, cp_220617_bag, x_tf_train_220617_bag)

(500, 6)
Creando el modelo...


2024-05-15 15:54:57.364624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 970 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:31:00.0, compute capability: 8.6
2024-05-15 15:54:57.367026: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 40933 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:98:00.0, compute capability: 8.6
2024-05-15 15:54:57.531118: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 8)                 480       
                                                                 
 flatten (Flatten)           (None, 8)                 0         
                                                                 
 dense (Dense)               (None, 8)                 72        
                                                                 
Total params: 552 (2.16 KB)
Trainable params: 552 (2.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
print("Entrenando el modelo...")
model_220617_bag.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model_220617_bag.fit(tf.constant(x_train_220617_bag), tf.cast(tf.constant(y_train_220617_bag), tf.float32), epochs=10, callbacks=[cp_220617_bag])

Entrenando el modelo...
Epoch 1/10


2024-05-15 15:55:11.132474: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7879f5bee0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-15 15:55:11.132521: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2024-05-15 15:55:11.132529: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA A40, Compute Capability 8.6
2024-05-15 15:55:11.144375: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-15 15:55:11.203561: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1715788511.327536  973820 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


92/92 [==============================] - 58s 593ms/step - loss: 1.8354 - accuracy: 0.0606
Epoch 2/10
92/92 [==============================] - 55s 593ms/step - loss: 1.6463 - accuracy: 0.4942
Epoch 3/10
92/92 [==============================] - 54s 589ms/step - loss: 1.5095 - accuracy: 0.5366
Epoch 4/10
92/92 [==============================] - 54s 590ms/step - loss: 1.4108 - accuracy: 0.5366
Epoch 5/10
92/92 [==============================] - 55s 598ms/step - loss: 1.3393 - accuracy: 0.5366
Epoch 6/10
92/92 [==============================] - 55s 595ms/step - loss: 1.2862 - accuracy: 0.5366
Epoch 7/10
92/92 [==============================] - 55s 596ms/step - loss: 1.2459 - accuracy: 0.5366
Epoch 8/10
92/92 [==============================] - 55s 593ms/step - loss: 1.2158 - accuracy: 0.5366
Epoch 9/10
92/92 [==============================] - 54s 590ms/step - loss: 1.1931 - accuracy: 0.5366
Epoch 10/10
92/92 [==============================] - 54s 589ms/step - loss: 1.1751 - accuracy: 0.5366


In [7]:
evals = model_220617_bag.evaluate(tf.constant(x_test_220617_bag), tf.cast(tf.constant(y_test_220617_bag), tf.float32))

23/23 [==============================] - 3s 118ms/step - loss: 1.7927 - accuracy: 0.5732


In [9]:
model_220617_bag.save('LSTMMemo/lstm.h5')